#### Part 2 Instructions
#### Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import os
import time
import json
from datetime import datetime
# Import the API key.
from config import g_key
from scipy.stats import linregress


In [2]:
file_name = os.path.join(".","data","WeatherPy_database.csv")
city_data_df = pd.read_csv(file_name)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Bredasdorp,ZA,-34.53,20.04,55.40,87,0,6.76,clear sky,0,0
1,1,Hobart,AU,-42.88,147.33,46.00,93,20,5.82,light rain,0,0
2,2,Muros,ES,42.78,-9.06,62.01,88,45,1.99,scattered clouds,0,0
3,3,Baruun-Urt,MN,46.68,113.28,68.47,65,99,13.40,overcast clouds,0,0
4,4,Hilo,US,19.73,-155.09,82.40,58,90,4.70,overcast clouds,0,0


In [3]:
city_data_df.count()

City_ID                570
City                   570
Country                568
Lat                    570
Lng                    570
Max Temp               570
Humidity               570
Cloudiness             570
Wind Speed             570
Current Description    570
Rain (inches)          570
Snow (inches)          570
dtype: int64

In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# Asking raning
raining = input("Do you want it to be raining? (yes/no) ")

Do you want it to be raining? (yes/no) no


In [6]:
# Ask snowing
snowing  = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be snowing? (yes/no) no


In [7]:
# Filter the dataset to find the cities that fit the criteria.
if raining.lower() == 'no' and snowing.lower() == 'no':
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] == 0.0) & \
                                           (city_data_df["Snow (inches)"] == 0.0) ]
elif raining.lower() == 'no' and snowing.lower() == 'yes':
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] == 0.0) & \
                                           (city_data_df["Snow (inches)"] > 0.0) ]
elif raining.lower() == 'yes' and snowing.lower() == 'no':
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] > 0.0) & \
                                           (city_data_df["Snow (inches)"] == 0.0) ]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] > 0) & \
                                           (city_data_df["Snow (inches)"] > 0) ]



preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
4,4,Hilo,US,19.73,-155.09,82.40,58,90,4.70,overcast clouds,0,0
5,5,Butaritari,KI,3.07,172.79,82.92,72,91,11.32,light rain,0,0
11,11,Kieta,PG,-6.22,155.63,85.69,63,7,1.61,clear sky,0,0
12,12,Bambanglipuro,ID,-7.95,110.28,78.13,83,98,6.35,overcast clouds,0,0
13,13,Avarua,CK,-21.21,-159.78,75.20,57,0,9.17,clear sky,0,0
14,14,Daur,PK,26.46,68.32,87.80,66,0,11.41,haze,0,0
16,16,Hurghada,EG,27.26,33.81,84.20,25,0,14.99,clear sky,0,0
20,20,Topeka,US,39.05,-95.68,75.20,73,90,3.36,overcast clouds,0,0
22,22,Statesboro,US,32.45,-81.78,82.40,58,75,5.82,broken clouds,0,0
23,23,Atuona,PF,-9.80,-139.03,80.44,75,2,12.80,clear sky,0,0


In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hilo,US,82.40,overcast clouds,19.73,-155.09,
5,Butaritari,KI,82.92,light rain,3.07,172.79,
11,Kieta,PG,85.69,clear sky,-6.22,155.63,
12,Bambanglipuro,ID,78.13,overcast clouds,-7.95,110.28,
13,Avarua,CK,75.20,clear sky,-21.21,-159.78,
14,Daur,PK,87.80,haze,26.46,68.32,
16,Hurghada,EG,84.20,clear sky,27.26,33.81,
20,Topeka,US,75.20,overcast clouds,39.05,-95.68,
22,Statesboro,US,82.40,broken clouds,32.45,-81.78,
23,Atuona,PF,80.44,clear sky,-9.80,-139.03,


In [9]:
# URL for googgle api
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters"

In [10]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # print(f'Index:{index}, Row:{row}')
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    try:
        # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping., index:{index}")

Hotel not found... skipping., index:11
Hotel not found... skipping., index:91
Hotel not found... skipping., index:92
Hotel not found... skipping., index:93
Hotel not found... skipping., index:107
Hotel not found... skipping., index:115
Hotel not found... skipping., index:186
Hotel not found... skipping., index:212
Hotel not found... skipping., index:256
Hotel not found... skipping., index:307
Hotel not found... skipping., index:317
Hotel not found... skipping., index:342
Hotel not found... skipping., index:344
Hotel not found... skipping., index:355
Hotel not found... skipping., index:383
Hotel not found... skipping., index:527
Hotel not found... skipping., index:548
Hotel not found... skipping., index:561


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hilo,US,82.40,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
5,Butaritari,KI,82.92,light rain,3.07,172.79,Isles Sunset Lodge
11,Kieta,PG,85.69,clear sky,-6.22,155.63,
12,Bambanglipuro,ID,78.13,overcast clouds,-7.95,110.28,Puri Brata
13,Avarua,CK,75.20,clear sky,-21.21,-159.78,Paradise Inn
14,Daur,PK,87.80,haze,26.46,68.32,Ramzan malik home
16,Hurghada,EG,84.20,clear sky,27.26,33.81,Sea Star Beau Rivage Hotel
20,Topeka,US,75.20,overcast clouds,39.05,-95.68,Capitol Plaza Hotel Topeka
22,Statesboro,US,82.40,broken clouds,32.45,-81.78,Holiday Inn Statesboro-University Area
23,Atuona,PF,80.44,clear sky,-9.80,-139.03,Villa Enata


In [14]:
# save dataframe to file
file_name = os.path.join(".", "data", "WeatherPy_vacation.csv")
hotel_df.to_csv(file_name, index_label="City_ID")

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a marker layer for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# hotel marker with names
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))